# German E-bay Car Listings Analysis

This project will clean and translate a dataset of German E-bay car listings, then draw conclusions about how the average price, mileage, and value loss from damage vary by brand.

The dataset and documentation can be found [here](https://data.world/data-society/used-cars-data).

## Reading the dataset file

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("datasets/autos.csv", encoding="Latin-1")

### Initial observations

In [2]:
autos

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,0,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,0,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,0,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,0,35683,2016-04-05 16:45:07


As the dataset is from a German website, many of the terms used are in German, such as "manuell" and "automatik" instead of "manual" and "automatic" for the `gearbox` column. The same applies to the `vehicleType` column, with entries such as "kleinwagen" and "kombri". The `price` and `odometer` columns need to be normalized to the integer type. The `notRepairedDamage` column uses "nein" and "ja" ("no" and "yes"), and should be converted into a boolean.

Column names use camel case instead of the preferred snake case.

### Checking for null values

In [3]:
for col in autos:
    if autos[col].isnull().any():
        print(f"{col} has null values.")

vehicleType has null values.
gearbox has null values.
model has null values.
fuelType has null values.
notRepairedDamage has null values.


Additionally, several columns contain null values. We can deal with these later, after cleaning up the columns.

## Cleaning the data

### Renaming and retyping columns

First, we will rename the columns in bulk to use snake case.

In [4]:
column_rename = {
    "dateCrawled": "ad_crawled",
    "offerType": "offer_type",
    "abtest": "ab_test",
    "vehicleType": "vehicle_type",
    "yearOfRegistration": "registration_year",
    "powerPS": "power_ps",
    "monthOfRegistration": "registration_month",
    "fuelType": "fuel_type",
    "notRepairedDamage": "unrepaired_damage",
    "dateCreated": "ad_created",
    "nrOfPictures": "picture_count",
    "postalCode": "postal_code",
    "lastSeen": "last_seen",
}

autos.rename(columns=column_rename, inplace=True)

autos.head(5)

,ad_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,picture_count,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The columns are now in a more manageable form. Next, we will see if any columns are not useful, like a text column where almost all values are the same.

In [5]:
autos.describe(include="all")

,ad_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,picture_count,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The following columns need some kind of change:

* `seller`, `offer_type`, `picture_count` - Not enough variance to be useful. Delete.
* `price`, `odometer` - Convert to an integer. Move units to column name.
* `unrepaired_damage` - Convert to a boolean.

We will also apply a set of translations to our data once these changes are in place.

In [6]:
autos.drop(["seller", "offer_type", "picture_count"], axis=1, inplace=True)

autos["price"] = (autos["price"]
                  .str.replace("$", "", regex=False) # Suppress a warning about regex
                  .str.replace(",", "")
                  .astype(int))
autos.rename(columns={"price": "price_usd"}, inplace=True)

autos["odometer"] = (autos["odometer"]
                     .str.replace("km", "")
                     .str.replace(",", "")
                     .astype(int))
autos.rename(columns={"odometer": "odometer_km"}, inplace=True)

autos["unrepaired_damage"] = (autos["unrepaired_damage"]
                            .str.replace("nein", "") # Empty strings are falsy
                            .astype(bool))

autos.describe(include="all")

,ad_crawled,name,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
count,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,50000,50000,50000.000000,50000
unique,48213,38754,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,False,2016-04-03 00:00:00,NaN,2016-04-07 06:17:27
freq,3,78,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,8
mean,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,50813.627300,NaN
std,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,25779.747957,NaN
min,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,30451.000000,NaN
50%,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,49577.000000,NaN
75%,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,71540.000000,NaN


### Cleaning bad values

The column names and types have been cleaned up. Now, we want to translate some remaining values and identify anomalous data.

#### Price

A price of 0 or null can be assumed faulty and removed before further pruning.

In [7]:
autos = autos.loc[autos["price_usd"] > 0 & ~autos["price_usd"].isnull()]

autos["price_usd"].describe()

count    4.857900e+04
mean     1.012788e+04
std      4.880873e+05
min      1.000000e+00
25%      1.200000e+03
50%      3.000000e+03
75%      7.490000e+03
max      1.000000e+08
Name: price_usd, dtype: float64

Now, we can remove outlier prices.

In [8]:
autos[autos["price_usd"] > 100_000].shape[0] / autos.shape[0] * 100

0.10910064019432265

Only 0.1% of cars cost over \$100,000. This seems like a good cutoff point to discard outliers.

In [9]:
autos[autos["price_usd"] < 1_000].shape[0] / autos.shape[0] * 100

20.45945779040326

While just over 20% of cars cost less than \$1,000, these are not good candidates for analysis, as this category will be overwhelmingly composed of "junkers", cars being sold as repair projects or for scrap, which we are not concerned with.

In [10]:
autos = autos[autos["price_usd"].between(1_000, 100_000)]
autos["price_usd"].describe()

count    38587.000000
mean      7089.045482
std       7893.814349
min       1000.000000
25%       2200.000000
50%       4300.000000
75%       8950.000000
max      99900.000000
Name: price_usd, dtype: float64

Now, prices average about \\$7,000. The data is still somewhat skewed to the high end, with the 50\% mark being only about \$4,500, but this is a more accurate image than what we had before, and more likely to represent the actual distribution of prices.

#### Odometer

Next, we will examine the `odometer_km` column for anything unusual.

In [11]:
autos["odometer_km"].describe()

count     38587.000000
mean     122859.771426
std       40712.638070
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

The odometer values seem normal. They range from 5,000 to 150,000. No action needs to be taken.

#### Registration years

In [12]:
autos["registration_year"].value_counts().sort_values(ascending=False).tail(10)

8888    1
1938    1
2800    1
1939    1
5911    1
1953    1
1951    1
1001    1
2019    1
9000    1
Name: registration_year, dtype: int64

In [13]:
autos["registration_year"].value_counts().sort_values(ascending=False).head(10)

2005    2785
2006    2651
2004    2608
2003    2477
2007    2258
2008    2202
2002    2135
2009    2076
2001    2064
2000    2000
Name: registration_year, dtype: int64

There are some very old cars for sale, but some of these use a year that is too low or high to be possible. The Model T started production in 1908, so any years before then would be good candidates, but we will check before deciding on a cutoff year to be conservative with potential historical cars.

In [14]:
autos[autos["registration_year"].between(1900, 1930)]

,ad_crawled,name,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
21416,2016-03-12 08:36:21,Essex_super_six__Ford_A,16500,control,cabrio,1927,manuell,40,andere,5000,5,benzin,ford,True,2016-03-12 00:00:00,74821,2016-03-15 12:45:12
22101,2016-03-09 16:51:17,BMW_Andere,11500,test,cabrio,1929,manuell,15,andere,5000,1,NaN,bmw,True,2016-03-09 00:00:00,70569,2016-04-07 06:17:11


We can see that the oldest car present with a reasonable year is an Essex Super Six from 1927. We will use this as our lower bound. The most recent ads in the data set are in 2016, so any years after that can be discarded.

In [15]:
autos = autos[autos["registration_year"].between(1927, 2016)]
autos["registration_year"].describe()

count    37167.000000
mean      2003.967552
std          7.008246
min       1927.000000
25%       2001.000000
50%       2005.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

We can see now that the distribution tends heavily towards cars produced after 2001, with 75% of all cars being sold falling into this category. Only 25% of cars on sale have registration years between 1927 and 2000.

#### Model

Some models are a null value. We will simply fill in with "unknown", as this information is not terribly important to our analysis, and discarding rows without a model would cut into relevant data.

In [16]:
autos["model"].fillna("unknown", inplace=True)
autos["model"].value_counts()

golf          2918
andere        2665
3er           2285
unknown       1429
passat        1152
              ... 
rangerover       1
200              1
b_max            1
samara           1
lanos            1
Name: model, Length: 242, dtype: int64

### Translating from German to English

The `vehicle_type`, `gearbox`, and `fuel_type` columns are currently in German. We will translate the values to make more sense of them.

#### Vehicle type

First, we will translate `vehicle_type`. This column contains some null values, which we will simply set to "other".

In [17]:
autos["vehicle_type"].fillna("other", inplace=True)
autos["vehicle_type"].unique()

array(['bus', 'limousine', 'kleinwagen', 'kombi', 'coupe', 'suv',
       'cabrio', 'other', 'andere'], dtype=object)

In [18]:
translate_vehicle_type = {
    "kleinwagen": "sedan",
    "kombi": "stationwagon",
    "cabrio": "convertible",
    "andere": "other",
}
autos["vehicle_type"].replace(translate_vehicle_type, inplace=True)
autos["vehicle_type"].value_counts()

limousine       10544
stationwagon     7532
sedan            6844
bus              3704
convertible      2840
coupe            2153
suv              1916
other            1634
Name: vehicle_type, dtype: int64

#### Gearbox

Next, we will translate the `gearbox` column to use "manual" and "automatic", and fill in null values with "unknown".

In [19]:
autos["gearbox"].fillna("unknown", inplace=True)
autos["gearbox"].unique()

array(['manuell', 'automatik', 'unknown'], dtype=object)

In [20]:
translate_gearbox = {
    "manuell": "manual",
    "automatik": "automatic",
}

autos["gearbox"].replace(translate_gearbox, inplace=True)
autos["gearbox"].value_counts(normalize=True)

manual       0.720316
automatic    0.245487
unknown      0.034197
Name: gearbox, dtype: float64

This data lines up with trends in the European car market. While automatic transmissions become a larger majority year over year in America, manual transmissions remain the dominant transmission type in Europe, with 72% of the cars in this dataset using a manual transmission.

#### Fuel type

The fuel types must also be translated. Though "benzin" translates directly to "petrol", we will use the american "gasoline". We will also unroll acronyms for clarity.

In [21]:
autos["fuel_type"].fillna("unknown", inplace=True)
autos["fuel_type"].value_counts()

benzin     21302
diesel     13354
unknown     1833
lpg          560
cng           59
hybrid        37
elektro       15
andere         7
Name: fuel_type, dtype: int64

In [22]:
translate_fuel_type = {
    "benzin": "gasoline",
    "lpg": "liquid petroleum gas",
    "cng": "compressed natural gas",
    "elektro": "electric",
    "andere": "other",
}

autos["fuel_type"].replace(translate_fuel_type, inplace=True)
autos["fuel_type"].value_counts()

gasoline                  21302
diesel                    13354
unknown                    1833
liquid petroleum gas        560
compressed natural gas       59
hybrid                       37
electric                     15
other                         7
Name: fuel_type, dtype: int64

#### Model

We have already filled null values in with "unknown", but "andere" is still present in the data set. We will translate this to "other".

In [23]:
autos["model"].replace({"andere": "other"}, inplace=True)
autos["model"].value_counts()

golf          2918
other         2665
3er           2285
unknown       1429
passat        1152
              ... 
rangerover       1
200              1
b_max            1
samara           1
lanos            1
Name: model, Length: 242, dtype: int64

In [24]:
if all(autos[col].notnull().all() for col in autos):
    print("All null values have been removed")

All null values have been removed


We have eliminated all null values.

## Analysis

Now that the data has been sufficiently cleaned, we can begin analyzing it to draw conclusions.

### Price by brand

First, we will compare average prices by brand. We will only be comparing brands considered "relevant" - those which make up at least 5% of the dataset.

In [25]:
brand_pcts = autos.value_counts("brand", normalize=True) * 100
relevant_brands = [ brand
                  for brand in autos["brand"].unique()
                  if brand_pcts[brand] > 5 ]

relevant_brands

['bmw', 'volkswagen', 'ford', 'audi', 'mercedes_benz', 'opel']

We will establish a new table, `autos_rel`, containing only cars of these brands.

In [26]:
autos_rel = autos[autos["brand"].isin(relevant_brands)]
autos_rel

,ad_crawled,name,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatic,286,7er,150000,6,gasoline,bmw,False,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manual,102,golf,70000,7,gasoline,volkswagen,False,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,stationwagon,2003,manual,0,focus,150000,7,gasoline,ford,False,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,1990,control,limousine,1998,manual,90,golf,150000,12,diesel,volkswagen,False,2016-03-16 00:00:00,53474,2016-04-07 03:17:32
13,2016-03-23 10:48:32,Audi_A3_1.6_tuning,1350,control,limousine,1999,manual,101,a3,150000,11,gasoline,audi,False,2016-03-23 00:00:00,12043,2016-04-01 14:17:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,2016-03-22 17:36:42,Audi_A6__S6__Avant_4.2_quattro_eventuell_Tausc...,5000,control,stationwagon,2001,automatic,299,a6,150000,1,gasoline,audi,False,2016-03-22 00:00:00,46537,2016-04-06 08:16:39
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,24900,control,limousine,2011,automatic,239,q5,100000,1,diesel,audi,False,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,1980,control,convertible,1996,manual,75,astra,150000,5,gasoline,opel,False,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,22900,control,stationwagon,2013,manual,150,a3,40000,11,diesel,audi,False,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


In [27]:
mean_brand_price = pd.Series(
    { brand: (autos_rel[autos_rel["brand"] == brand]
        ["price_usd"].mean())
    for brand in autos_rel["brand"].unique() }
)

mean_brand_price

bmw               8975.103840
volkswagen        6645.132602
ford              5274.369217
audi             10276.903581
mercedes_benz     9196.060738
opel              4219.954737
dtype: float64

As we can see from the collection of mean brand prices:

* Audi, BMW, and Mercedes-Benz are the most expensive.
* Volkswagen is mid-priced
* Ford and Opel are less expensive.

We can now convert out mean brand price series into a dataframe to collect more brand information.

In [28]:
brands = pd.DataFrame(mean_brand_price, columns=["mean_price"])
brands

,mean_price
bmw,8975.103840
volkswagen,6645.132602
ford,5274.369217
audi,10276.903581
mercedes_benz,9196.060738
opel,4219.954737


In [29]:
high_end = [ "audi", "mercedes_benz", "bmw" ]
high_end_mean_price = brands.loc[high_end, "mean_price"].mean()

high_end_mean_price

9482.689386390644

In [30]:
low_end = [ "ford", "opel" ]
low_end_mean_price = brands.loc[low_end, "mean_price"].mean()

low_end_mean_price

4747.161977075835

In [31]:
low_end_mean_price / high_end_mean_price * 100

50.061346350634054

If we again average the high-end cars and low-end cars as groups, we find that the average low-end car costs half as much as the average high-end car.

### Mileage by brand

Next, we will examine if the mean mileage of cars differs by brand.

In [32]:
brands["mean_odometer_km"] = pd.Series(
    { brand: (autos_rel.loc[autos_rel["brand"] == brand]
        ["odometer_km"].mean())
     for brand in brands.index.values }
)

brands["mean_odometer_km"]

bmw              132068.225703
volkswagen       125765.650899
ford             119651.855245
audi             127524.793388
mercedes_benz    130130.151844
opel             123952.926976
Name: mean_odometer_km, dtype: float64

The variation in odometer kilometers by brand is slight, but there is a pattern of higher-end cars being sold with higher mileage, and lower-end cars being sold with lower mileage.

### Most common model by brand

We can also check what model is most common for each brand and rename some to be more clear.

In [33]:
brands["top_model"] = pd.Series(
    { brand: (autos_rel.loc[autos_rel["brand"] == brand]
        ["model"].mode()[0])
    for brand in brands.index.values }
)

model_rename = {
    "c_klasse": "c_class",
    "3er": "3_series",
}
brands["top_model"].replace(model_rename, inplace=True)

brands["top_model"]

bmw              3_series
volkswagen           golf
ford                focus
audi                   a4
mercedes_benz     c_class
opel                astra
Name: top_model, dtype: object

### Most common vehicle type by brand

With the most common model identified, we will next find the most common vehicle type for each brand.

In [34]:
brands["top_vehicle_type"] = pd.Series(
    { brand: (autos_rel.loc[autos_rel["brand"] == brand]
        ["vehicle_type"].mode()[0])
    for brand in brands.index.values }
)

brands["top_vehicle_type"]

bmw                 limousine
volkswagen          limousine
ford                    sedan
audi             stationwagon
mercedes_benz       limousine
opel                    sedan
Name: top_vehicle_type, dtype: object

We 

### Percent of manual transmissions per brand

We already know that 75% of the car listings are for manuals, but we would also like to know how the distribution breaks down by brand.

In [35]:
brands["pct_manual"] = pd.Series(
    { brand: (autos_rel.loc[autos_rel["brand"] == brand]
        ["gearbox"].value_counts(normalize=True)
        .sort_index()[1] * 100)
    for brand in brands.index.values }
)

brands["pct_manual"]

bmw              62.046771
volkswagen       80.836415
ford             87.081997
audi             61.515152
mercedes_benz    40.009641
opel             85.455643
Name: pct_manual, dtype: float64

A clear pattern emerges: higher-end cars are more likely to be automatic, with lower-end cars being more likely to be manual. This could be because the abundance of manual transmissions in Europe causes automatics to be more rare, and thus more valuable, causing a higher-end car to be more "worth" the extra cost of an automatic transmission.

Notably, Mercedez-Benz cars are significantly more likely to be automatic, despite not being as expensive as Audis on average. This is likely a simple difference in brand priorities, with Mercedez-Benz emphasizing the "luxury" of an automatic.

### Average price based on unrepaired damage

We are interested in learning how much unrepaired damage affects the price of a vehicle by brand.

In [36]:
brands["mean_price_damaged"] = pd.Series(
    { brand: (autos_rel.loc[(autos_rel["brand"] == brand)
            & (autos_rel["unrepaired_damage"])]
        ["price_usd"].mean())
    for brand in brands.index.values }
)

brands["mean_price_damaged"]

bmw              5747.610738
volkswagen       4190.473401
ford             3574.842920
audi             5753.237500
mercedes_benz    5735.743405
opel             2951.419492
Name: mean_price_damaged, dtype: float64

In [37]:
brands["mean_price_undamaged"] = pd.Series(
    { brand: (autos_rel.loc[(autos_rel["brand"] == brand)
            & (~autos_rel["unrepaired_damage"])]
        ["price_usd"].mean())
    for brand in brands.index.values }
)

brands["mean_price_undamaged"]

bmw               9741.065835
volkswagen        7310.715235
ford              5718.150780
audi             11396.161168
mercedes_benz    10066.620211
opel              4564.591328
Name: mean_price_undamaged, dtype: float64

In [38]:
brands["damaged_price_pct"] = (brands["mean_price_damaged"]
                                / brands["mean_price_undamaged"]
                                * 100)

brands["damaged_price_pct"]

bmw              59.003920
volkswagen       57.319609
ford             62.517465
audi             50.483996
mercedes_benz    56.977846
opel             64.659008
Name: damaged_price_pct, dtype: float64

Again, a clear pattern based on the brand emerges: More expensive cars lose more value from having unrepaired damage, with Audis retaining only 50% value compared to Fords retaining 62% value. This makes sense because more expensive cars are typically more expensive to repair, and less expensive cars are often built to be more durable and can withstand some damage. A high-end car also loses some of its desirability if it is damaged.

### Overview by brand

We will now take a broad review our brand analysis.

In [39]:
brands

,mean_price,mean_odometer_km,top_model,top_vehicle_type,pct_manual,mean_price_damaged,mean_price_undamaged,damaged_price_pct
bmw,8975.103840,132068.225703,3_series,limousine,62.046771,5747.610738,9741.065835,59.003920
volkswagen,6645.132602,125765.650899,golf,limousine,80.836415,4190.473401,7310.715235,57.319609
ford,5274.369217,119651.855245,focus,sedan,87.081997,3574.842920,5718.150780,62.517465
audi,10276.903581,127524.793388,a4,stationwagon,61.515152,5753.237500,11396.161168,50.483996
mercedes_benz,9196.060738,130130.151844,c_class,limousine,40.009641,5735.743405,10066.620211,56.977846
opel,4219.954737,123952.926976,astra,sedan,85.455643,2951.419492,4564.591328,64.659008


As car brands become more expensive, they:

* Increase in average mileage.
* Increase in likelihood of being an automatic transmission.
* Increase in value lost from unrepaired damage.
* Tend to be limousines instead of sedans.

## Clean dataset files

The cleaned `autos` and `autos_rel` tables are available in `datasets/`, as created below.

In [40]:
autos.to_csv("datasets/autos_cleaned.csv")
autos.to_csv("datasets/autos_rel.csv")